# SEED Algorithm: Spatial Economic and Environmental Distribution Algorithm

**Objective:** To select and distribute 1000 optimal locations for senior living facilities in Spain, maximizing territorial coverage, economic viability, and social demand.

---

### Algorithm architecture (4 layers):

1. **Layer 1 - Territorial Base**: Decision space (36,000 census tracts)
2. **Layer 2 - Residential Demand**: Number of Households + Dependency + Density (weight 0.45)
3. **Layer 3 - Economic Viability**: Average Household Income (weight 0.40)
4. **Layer 4 - Saturation**: Territorial Correction Factor (weight 0.15)

### SEED formula:

```
SEED = 0.45*(0.65*number of households + 0.1*dependency + 0.25*density) + 0.4*income + 0.15*saturation
```

### Spatial constraint: Clustering
- Minimum distance between residences: **variable distance by population density**
- Iterative selection from highest score

---

## Install dependencies

In [1]:
try:
    import pandas as pd
    import numpy as np
    import folium
    from scipy.spatial.distance import cdist
    from sklearn.preprocessing import MinMaxScaler
    import matplotlib.pyplot as plt
    from matplotlib.colors import LinearSegmentedColormap
    from folium.plugins import HeatMap
    import branca.colormap as cm
except ImportError:
    !pip install pandas numpy folium scipy scikit-learn openpyxl matplotlib -q

## Upload and prepare data

In [2]:
import pandas as pd
import numpy as np
import folium
from folium import plugins
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
from pathlib import Path

ARCHIVO_DATOS = Path("/home/lgarbayo/Escritorio/mvp-residence/data/VARIABLES SEED.xlsx")
DISTANCIA_MIN_CIUDAD_GRANDE = 1.5  # density > 5000 hab/km²
DISTANCIA_MIN_CIUDAD_MEDIA = 2.5   # density > 1000 hab/km²
DISTANCIA_MIN_RURAL = 5.0    
NUM_RESIDENCIAS = 1000

# output directory
output_dir = Path('outputs')
output_dir.mkdir(exist_ok=True)

if not Path(ARCHIVO_DATOS).exists():
    raise FileNotFoundError(f"File not found: {ARCHIVO_DATOS}")

df = pd.read_excel(ARCHIVO_DATOS, sheet_name=0)
df = df.rename(columns={
    'id_seccion': 'seccion_censal',
    'length': 'longitud',
    'latitude': 'latitud'
})

## Public functions

In [3]:
def exportar_capa_csv(df, nombre_capa, output_dir):
    """
    export descending csv filtered by score
    """
    filename = output_dir / f"{nombre_capa}_ranking.csv"
    
    # select interest column
    columnas_base = ['seccion_censal', 'latitud', 'longitud', 'nombre_provincia']
    columnas_scores = [col for col in df.columns if 'score' in col.lower() or 'norm' in col]
    columnas_export = columnas_base + columnas_scores + ['f_of_m', 'density', 'dependence', 'rent', 'saturation']
    columnas_export = [c for c in columnas_export if c in df.columns]

    df_export = df[columnas_export].copy()
    df_export['seccion_censal'] = df_export['seccion_censal'].astype(str).str.zfill(10)
    
    df_export.to_csv(filename, index=False, encoding='utf-8')

def crear_mapa_gradiente(df, score_column, titulo, filename, incluir_canarias=True):
    """
    df: DataFrame with data
    score_column: Name of the column with the score
    titulo: Map title
    filename: Output file name
    incluir_canarias: If True, adjusts the map to include the Canary Islands
    """
    
    # use all sections for the gradient
    df_top = df.copy()
    df_top['latitud'] = pd.to_numeric(df_top['latitud'], errors='coerce')
    df_top['longitud'] = pd.to_numeric(df_top['longitud'], errors='coerce')
    df_top[score_column] = pd.to_numeric(df_top[score_column], errors='coerce')
    df_top = df_top.dropna(subset=['latitud', 'longitud', score_column])
    
    # adjust center to include islands
    if incluir_canarias:
        centro_lat = 38.0
        centro_lon = -5.0
        zoom = 5
    else:
        centro_lat = 40.4168
        centro_lon = -3.7038
        zoom = 6
    
    mapa = folium.Map(
        location=[centro_lat, centro_lon],
        zoom_start=zoom,
        tiles='OpenStreetMap'
    )
    
    titulo_html = f'''
    <div style="position: fixed; top: 10px; left: 50px; width: 600px; 
                background-color: white; border: 2px solid #2C3E50; z-index: 9999;
                padding: 15px; border-radius: 10px; font-family: Arial;">
        <h3 style="margin: 0; color: #2C3E50;">{titulo}</h3>
    </div>
    '''
    mapa.get_root().html.add_child(folium.Element(titulo_html))
    
    # ranking-based weights (top rank -> hottest)
    df_top = df_top.sort_values(score_column, ascending=False).reset_index(drop=True)
    n_rows = len(df_top)
    if n_rows > 1:
        df_top['heat_weight'] = 1 - (df_top.index / (n_rows - 1))
    else:
        df_top['heat_weight'] = 1.0

    # keep only top 5% by ranking position
    if n_rows > 0:
        cutoff = max(1, int(n_rows * 0.05))
        df_top = df_top.iloc[:cutoff].copy()

    heat_data = []
    for _, row in df_top.iterrows():
        weight = row['heat_weight'] ** 2
        if not np.isfinite(weight):
            continue
        heat_data.append([row['latitud'], row['longitud'], weight])


    gradient = {
        0.0: '#00ff00',
        0.5: '#ffff00',
        1.0: '#ff0000'
    }

    folium.plugins.HeatMap(
        heat_data,
        radius=12,
        blur=18,
        max_zoom=10,
        gradient=gradient
    ).add_to(mapa)

    if not df_top.empty:
        bounds = [
            [df_top['latitud'].min(), df_top['longitud'].min()],
            [df_top['latitud'].max(), df_top['longitud'].max()]
        ]
        mapa.fit_bounds(bounds, padding=(20, 20))

    mapa.save(str(filename))
    return mapa

def calcular_distancia_adaptativa(densidad_hab_km2):
    """
    calculate minimum distance by density
    """
    if densidad_hab_km2 > 5000:
        return DISTANCIA_MIN_CIUDAD_GRANDE
    elif densidad_hab_km2 > 1000:
        return DISTANCIA_MIN_CIUDAD_MEDIA
    else:
        return DISTANCIA_MIN_RURAL

## Layer no.1: territorial base

In [4]:
# filter sections with valid coordinates
df_valid = df.dropna(subset=['latitud', 'longitud']).copy()

# extract province from census section code
df_valid['provincia'] = df_valid['seccion_censal'].astype(str).str.zfill(10).str[:2]

# code provinces dictionary of INE
PROVINCIAS = {
    '01': 'Álava', '02': 'Albacete', '03': 'Alicante', '04': 'Almería',
    '05': 'Ávila', '06': 'Badajoz', '07': 'Baleares', '08': 'Barcelona',
    '09': 'Burgos', '10': 'Cáceres', '11': 'Cádiz', '12': 'Castellón',
    '13': 'Ciudad Real', '14': 'Córdoba', '15': 'A Coruña', '16': 'Cuenca',
    '17': 'Girona', '18': 'Granada', '19': 'Guadalajara', '20': 'Gipuzkoa',
    '21': 'Huelva', '22': 'Huesca', '23': 'Jaén', '24': 'León',
    '25': 'Lleida', '26': 'La Rioja', '27': 'Lugo', '28': 'Madrid',
    '29': 'Málaga', '30': 'Murcia', '31': 'Navarra', '32': 'Ourense',
    '33': 'Asturias', '34': 'Palencia', '35': 'Las Palmas', '36': 'Pontevedra',
    '37': 'Salamanca', '38': 'S.C. Tenerife', '39': 'Cantabria', '40': 'Segovia',
    '41': 'Sevilla', '42': 'Soria', '43': 'Tarragona', '44': 'Teruel',
    '45': 'Toledo', '46': 'Valencia', '47': 'Valladolid', '48': 'Bizkaia',
    '49': 'Zamora', '50': 'Zaragoza', '51': 'Ceuta', '52': 'Melilla'
}

df_valid['nombre_provincia'] = df_valid['provincia'].map(PROVINCIAS).fillna('Desconocida')

## Layer no.2: residential demand

### Normalization with scikit-learn

We use `scikit-learn.preprocessing.MinMaxScaler` to normalize all variables to the range [0, 1].

**MinMaxScaler Formula:**
```
X_scaled = (X - X_min) / (X_max - X_min)
```

**Inversion for negative variables:**
- F-of-M: smaller is better → invert (1 - X_scaled)
- Saturation: smaller is better → invert (1 - X_scaled)

In [5]:
scaler = MinMaxScaler()

# normalize and invert
f_of_m_normalized = scaler.fit_transform(df_valid[['f_of_m']])
df_valid['f_of_m_norm'] = 1 - f_of_m_normalized.flatten()

dependence_normalized = scaler.fit_transform(df_valid[['dependence']])
df_valid['dependence_norm'] = dependence_normalized.flatten()

density_normalized = scaler.fit_transform(df_valid[['density']])
df_valid['density_norm'] = density_normalized.flatten()

# calculate demand score
df_valid['score_demanda'] = (
    0.65 * df_valid['f_of_m_norm'] +
    0.10 * df_valid['dependence_norm'] +
    0.25 * df_valid['density_norm']
)

df_capa2 = df_valid.sort_values('score_demanda', ascending=False).reset_index(drop=True)
exportar_capa_csv(df_capa2, 'capa2_demanda', output_dir)
crear_mapa_gradiente(
    df_capa2,
    'score_demanda',
    'Layer no.2: residential demand',
    output_dir / 'capa2_demanda_mapa_gradiente.html',
    incluir_canarias=True
)

## Layer no.3: economic viability

In [6]:
def calcular_score_renta(renta):
    """
    calculate the economic viability score based on average income.
    asymmetric curve with an optimum at €72,000:
        stronger penalty below the optimum
        softer penalty above the optimum
    reference points:
        €40,000 → 0.00
        €72,000 → 1.00 (optimum)
    """
    RENTA_OPTIMA = 72000
    RENTA_MIN = 40000
    
    if renta <= RENTA_MIN:
        return 0.0
    elif renta <= RENTA_OPTIMA:
        # increasing curve towards optimum (steeper)
        # use quadratic function for steeper slope
        x = (renta - RENTA_MIN) / (RENTA_OPTIMA - RENTA_MIN)
        return x ** 0.7  # exp < 1 for rapid grow
    else:
        # smooth decreasing curve after the optimum, milder penalty
        x = (renta - RENTA_OPTIMA) / (RENTA_OPTIMA - RENTA_MIN)
        return 1.0 / (1 + 0.5 * x ** 1.5)

df_valid['score_renta'] = df_valid['rent'].apply(calcular_score_renta)
# acumulated score
df_valid['score_demanda_renta'] = (
    0.45 * df_valid['score_demanda'] +
    0.40 * df_valid['score_renta']
)

df_capa3_csv = df_valid.sort_values('score_demanda_renta', ascending=False).reset_index(drop=True)
df_capa3_mapa = df_valid.sort_values('score_renta', ascending=False).reset_index(drop=True)
exportar_capa_csv(df_capa3_csv, 'capa3_demanda_renta', output_dir)
exportar_capa_csv(df_capa3_mapa, 'capa3_solo_renta',output_dir)
crear_mapa_gradiente(
    df_capa3_mapa,
    'score_renta',
    'Layer no.3: economic viability',
    output_dir / 'capa3_renta_mapa_gradiente.html',
    incluir_canarias=True
)

## Layer no.4: territorial saturation

In [7]:
scaler_sat = MinMaxScaler()

# normalize and invert
saturation_normalized = scaler_sat.fit_transform(df_valid[['saturation']])
df_valid['saturation_norm'] = 1 - saturation_normalized.flatten()

df_capa4_mapa = df_valid.sort_values('saturation_norm', ascending=False).reset_index(drop=True)
exportar_capa_csv(df_capa4_mapa, 'capa4_solo_saturacion', output_dir)
crear_mapa_gradiente(
    df_capa4_mapa,
    'saturation_norm',
    'Layer no.4: territorial saturation',
    output_dir / 'capa4_saturacion_mapa_gradiente.html',
    incluir_canarias=True
)

## Final SEED score

In [8]:
# SEED = 0.45*(score_demanda) + 0.40*(score_renta) + 0.15*(score_saturacion)

df_valid['SEED_score'] = (
    0.45 * df_valid['score_demanda'] +
    0.40 * df_valid['score_renta'] +
    0.15 * df_valid['saturation_norm']
)

df_capa4_csv = df_valid.sort_values('SEED_score', ascending=False).reset_index(drop=True)
exportar_capa_csv(df_capa4_csv, 'seed_preclustering', output_dir)
crear_mapa_gradiente(
    df_capa4_csv,
    'SEED_score',
    'Final score pre-clustering',
    output_dir / 'seed_preclustering_mapa_gradiente.html',
    incluir_canarias=True
)

## Clustering

In [9]:
def calcular_distancia_haversine(lat1, lon1, lat2, lon2):
    """
    calculate the distance in km between two points using the Haversine formula.    
    """
    # earth ratio in km
    R = 6371.0
    
    # radians transform
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)
    
    # diff
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    # haversine formula
    a = np.sin(dlat/2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    
    return R * c


def seleccionar_con_clustering_adaptativo(df, num_residencias):
    """
    select locations to avoid territorial overlaps
    1. sort by descending seed score
    2. select the best section
    3. remove all sections less than distancia_min_km
    4. repeat until num_residencias is reached
    """
    
    # sort by descending score
    df_sorted = df.sort_values('SEED_score', ascending=False).reset_index(drop=True)
    
    seleccionadas = []
    coords_seleccionadas = []
    
    candidatos = df_sorted.copy()
    
    iteracion = 0
    
    while len(seleccionadas) < num_residencias and len(candidatos) > 0:
        iteracion += 1
        
        # select the best secion
        mejor = candidatos.iloc[0]
        seleccionadas.append(mejor)
        coords_seleccionadas.append((mejor['latitud'], mejor['longitud']))

        # calculate adapted distance
        densidad_actual = mejor.get('density', 0)
        distancia_min_km = calcular_distancia_adaptativa(densidad_actual)
        
        # only for logging
        if densidad_actual > 5000:
            tipo_zona = "Ciudad Grande"
        elif densidad_actual > 1000:
            tipo_zona = "Ciudad Media"
        else:
            tipo_zona = "Rural"
        
        if iteracion % 100 == 0 or iteracion <= 10:
            print(f"   • Iteración {iteracion:4d}: Sección {int(mejor['seccion_censal'])}, "
              f"Score: {mejor['SEED_score']:.4f}, Densidad: {densidad_actual:.1f} hab/km², "
              f"Distancia: {distancia_min_km:.1f}km ({tipo_zona}), Candidatos: {len(candidatos):,}")
        
        # remove selected
        candidatos = candidatos.iloc[1:].copy()
        
        if len(candidatos) == 0:
            break
        
        # calc distances from selected to all
        distancias = calcular_distancia_haversine(
            mejor['latitud'],
            mejor['longitud'],
            candidatos['latitud'].values,
            candidatos['longitud'].values
        )
        
        # filter cand with more distancia_min_km
        mask_validas = distancias >= distancia_min_km
        candidatos = candidatos[mask_validas].reset_index(drop=True)
    
    return pd.DataFrame(seleccionadas)


# play selection
df_seed_1000 = seleccionar_con_clustering_adaptativo(
    df_valid,
    NUM_RESIDENCIAS
)

# select top 1000
df_seed_1000['ranking'] = range(1, len(df_seed_1000) + 1)

   • Iteración    1: Sección 1503003001, Score: 0.8379, Densidad: 0.0 hab/km², Distancia: 5.0km (Rural), Candidatos: 36,333
   • Iteración    2: Sección 3501602037, Score: 0.8318, Densidad: 0.2 hab/km², Distancia: 5.0km (Rural), Candidatos: 36,144
   • Iteración    3: Sección 3605709011, Score: 0.8277, Densidad: 0.8 hab/km², Distancia: 5.0km (Rural), Candidatos: 35,917
   • Iteración    4: Sección 2006906001, Score: 0.8184, Densidad: 0.3 hab/km², Distancia: 5.0km (Rural), Candidatos: 35,896
   • Iteración    5: Sección 4625001009, Score: 0.8183, Densidad: 0.1 hab/km², Distancia: 5.0km (Rural), Candidatos: 35,746
   • Iteración    6: Sección 5100101001, Score: 0.8180, Densidad: 0.2 hab/km², Distancia: 5.0km (Rural), Candidatos: 35,106
   • Iteración    7: Sección 2906702017, Score: 0.8150, Densidad: 1.1 hab/km², Distancia: 5.0km (Rural), Candidatos: 35,050
   • Iteración    8: Sección 1505801005, Score: 0.8094, Densidad: 1.8 hab/km², Distancia: 5.0km (Rural), Candidatos: 34,990
   • Ite

## Export clustering Heatmap

In [10]:
crear_mapa_gradiente(
    df_seed_1000,
    'SEED_score',
    'Final score after algorithm training',
    output_dir / 'SEED_FINAL_post_clustering_mapa_gradiente.html',
    incluir_canarias=True
)

## Export CSV

In [11]:
df_top50 = df_seed_1000.head(50).copy()

columnas_export = [
    'ranking', 'seccion_censal', 'SEED_score',
    'latitud', 'longitud', 'nombre_provincia',
    'f_of_m', 'density', 'dependence', 'rent', 'saturation',
    'score_demanda', 'score_renta', 'saturation_norm'
]

output_top50 = output_dir / 'SEED_top50_ubicaciones.csv'
df_top50[columnas_export].to_csv(output_top50, index=False, encoding='utf-8')

output_top1000 = output_dir / 'SEED_top1000_ubicaciones.csv'
df_seed_1000[columnas_export].to_csv(output_top1000, index=False, encoding='utf-8')

## Map no.1: top 50

In [12]:
import folium

mapa_top50 = folium.Map(
    location=[40.4168, -3.7038],
    zoom_start=6,
    tiles='OpenStreetMap'
)

for _, row in df_top50.iterrows():
    rank = int(row['ranking'])
    
    # Color y estilo según ranking
    if rank <= 10:
        color = 'green'
        icon_type = 'star'
        emoji = '🥇'
    elif rank <= 25:
        color = 'blue'
        icon_type = 'home'
        emoji = '🔵'
    else:
        color = 'orange'
        icon_type = 'home'
        emoji = '🟠'
    
    popup_html = f"""
    <div style='font-family: Arial; font-size: 13px; min-width: 300px;'>
        <h3 style='margin:0 0 10px 0; color: #2C3E50;'>{emoji} Ranking #{rank}</h3>
        <hr style='margin: 10px 0; border: 1px solid #ddd;'>
        <table style='width:100%; border-collapse: collapse;'>
            <tr style='background-color: #f8f9fa;'>
                <td style='padding: 8px; font-weight: bold;'>Sección Censal:</td>
                <td style='padding: 8px;'>{int(row['seccion_censal'])}</td>
            </tr>
            <tr>
                <td style='padding: 8px; font-weight: bold;'>Score SEED:</td>
                <td style='padding: 8px;'><strong>{row['SEED_score']:.4f}</strong></td>
            </tr>
            <tr style='background-color: #f8f9fa;'>
                <td style='padding: 8px; font-weight: bold;'>Provincia:</td>
                <td style='padding: 8px;'>{row['nombre_provincia']}</td>
            </tr>
            <tr>
                <td style='padding: 8px; font-weight: bold;'>F-of-M:</td>
                <td style='padding: 8px;'>{row['f_of_m']:.4f}</td>
            </tr>
            <tr style='background-color: #f8f9fa;'>
                <td style='padding: 8px; font-weight: bold;'>Renta media:</td>
                <td style='padding: 8px;'>{row['rent']:,.0f}€</td>
            </tr>
            <tr>
                <td style='padding: 8px; font-weight: bold;'>Coordenadas:</td>
                <td style='padding: 8px;'>{row['latitud']:.4f}, {row['longitud']:.4f}</td>
            </tr>
        </table>
    </div>
    """
    
    folium.Marker(
        location=[row['latitud'], row['longitud']],
        popup=folium.Popup(popup_html, max_width=400),
        tooltip=f"{emoji} #{rank}: {row['nombre_provincia']} (SEED: {row['SEED_score']:.4f})",
        icon=folium.Icon(color=color, icon=icon_type, prefix='fa')
    ).add_to(mapa_top50)

output_mapa_top50 = output_dir / 'SEED_mapa_top50.html'
mapa_top50.save(str(output_mapa_top50))
mapa_top50

## Map no.2: top 1000

In [13]:
mapa_top1000 = folium.Map(
    location=[40.4168, -3.7038],
    zoom_start=6,
    tiles='OpenStreetMap'
)

for _, row in df_seed_1000.iterrows():
    # simplify popup
    popup_text = f"""
    <div style='font-family: Arial; font-size: 12px;'>
        <strong>Ranking #{int(row['ranking'])}</strong><br>
        Sección: {int(row['seccion_censal'])}<br>
        Score: {row['SEED_score']:.4f}<br>
        Provincia: {row['nombre_provincia']}
    </div>
    """
    
    folium.CircleMarker(
        location=[row['latitud'], row['longitud']],
        radius=4,
        popup=folium.Popup(popup_text, max_width=250),
        tooltip=f"#{int(row['ranking'])}: {row['nombre_provincia']}",
        color='darkred',
        fill=True,
        fillColor='red',
        fillOpacity=0.7,
        weight=2
    ).add_to(mapa_top1000)

output_mapa_top1000 = output_dir / 'SEED_mapa_top1000.html'
mapa_top1000.save(str(output_mapa_top1000))
mapa_top1000